In [1]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Acessa a base de sócios e administradores da Receita Federal
# Cruza com os dados de pagamentos da covid-19
# Separa empresas do Brasil que têm sócios os candidatos das eleições de 2018
# Verifica se um destes é sócio das empresas com pagamentos na covid-19
#

In [2]:
import pandas as pd
from datetime import datetime
import unidecode

In [3]:
# Função para retirar acentuação 
def f(str):
    return (unidecode.unidecode(str))

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
pagamentos_covid19 = pd.read_csv("resultados/pagamentos_covid19/pagamentos_feitos_de_empenhos_covid19_ate_15_outubro_2020.csv", **kwargs)

In [6]:
pagamentos_covid19.rename(columns = {'Id Empenho':'id_empenho'},inplace = True)
pagamentos_covid19.rename(columns = {'Código Empenho':'cod_empenho'},inplace = True)

pagamentos_covid19['Valor Pago (R$)'] = pagamentos_covid19['Valor Pago (R$)'].str.replace(',','.')
pagamentos_covid19['Valor Pago (R$)'] = pagamentos_covid19['Valor Pago (R$)'].astype('float64') 
pagamentos_covid19.rename(columns = {'Valor Pago (R$)':'valor_pago_em_real'},inplace = True)

In [7]:
# Procura o Código Favorecido na Receita Federal

In [7]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/setembro_2020/cnpj_dados_cadastrais_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'razao_social',
                                      'nome_fantasia',
                                      'situacao_cadastral',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio'])

In [8]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    empresas_encontradas = pd.merge(pagamentos_covid19.drop_duplicates('Código Favorecido_x'), \
                                 df, \
                                 left_on= 'Código Favorecido_x', \
                                 right_on='cnpj')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_procura1 = empresas_encontradas
        else:
            df_procura1 = df_procura1.append(empresas_encontradas)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-10-20 16:30:56.678602
A extração terminou às:  2020-10-20 16:50:26.605101


In [9]:
df_procura1.shape

(8038, 112)

In [10]:
# Onde estão a maioria das empresas

In [11]:
conta = df_procura1.groupby("municipio").id_empenho.count().reset_index()
conta.sort_values('id_empenho', ascending=False)

,municipio,id_empenho
1526,RIO DE JANEIRO,423
1731,SAO PAULO,373
265,BRASILIA,264
1434,PORTO ALEGRE,195
204,BELO HORIZONTE,184
...,...,...
1105,MATUPA,1
1104,MATRIZ DE CAMARAGIBE,1
1103,MATRINCHA,1
1102,MATOES DO NORTE,1


In [12]:
df_procura1.to_csv('resultados/pagamentos_covid19_com_empresas/pagamentos_feitos_de_empenhos_covid19_com_empresas_ate_15_outubro_2020.csv', index=False)

In [13]:
# Adiciona socios

In [15]:
# Porém com isso a soma com os valores não deve ser mais feita, porque cada linha é repetida sempre que mais de um sócio é encontrado

In [14]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/setembro_2020/cnpj_dados_socios_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'identificador_socio',
                                      'nome_socio',
                                      'cnpj_cpf_socio',
                                      'cod_qualificacao_socio',
                                      'data_entrada_sociedade',
                                      'cpf_representante_legal',
                                      'nome_representante'])

In [15]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    socios_encontrados = pd.merge(df_procura1, \
                                 df, \
                                 left_on='cnpj', \
                                 right_on='cnpj')
    
    if socios_encontrados.empty is False:
        if conta == 0:
            df_procura_final = socios_encontrados
        else:
            df_procura_final = df_procura_final.append(socios_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-10-20 16:51:16.371420
A extração terminou às:  2020-10-20 16:56:37.794870


In [16]:
df_procura_final.shape

(12449, 119)

In [17]:
df_procura_final.to_csv('resultados/pagamentos_covid19_com_empresas_e_socios/pagamentos_feitos_de_empenhos_covid19_com_empresas_e_socios_ate_15_outubro_2020.csv', index=False)

In [18]:
# Carrega candidatos de 2018 e 2016

In [19]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'latin_1'}
candidatos_2018 = pd.read_csv("../candidatos_2018/consulta_cand_2018_BRASIL.csv", **kwargs)

In [20]:
# Apaga as colunas que não serão usadas
del candidatos_2018['DT_GERACAO']
del candidatos_2018['HH_GERACAO']
#del candidatos_2018['ANO_ELEICAO']
del candidatos_2018['CD_TIPO_ELEICAO']
del candidatos_2018['NM_TIPO_ELEICAO']
del candidatos_2018['NR_TURNO']
del candidatos_2018['CD_ELEICAO']
del candidatos_2018['DS_ELEICAO']
del candidatos_2018['DT_ELEICAO']
del candidatos_2018['TP_ABRANGENCIA']
del candidatos_2018['SG_UE']
del candidatos_2018['NM_UE']
del candidatos_2018['CD_CARGO']
del candidatos_2018['CD_SITUACAO_CANDIDATURA']
del candidatos_2018['CD_DETALHE_SITUACAO_CAND']
del candidatos_2018['DS_DETALHE_SITUACAO_CAND']
del candidatos_2018['TP_AGREMIACAO']
del candidatos_2018['NR_PARTIDO']
del candidatos_2018['NM_PARTIDO']
del candidatos_2018['SQ_COLIGACAO']
del candidatos_2018['NM_COLIGACAO']
del candidatos_2018['DS_COMPOSICAO_COLIGACAO']
del candidatos_2018['CD_NACIONALIDADE']
#del candidatos_2018['DS_NACIONALIDADE']
#del candidatos_2018['SG_UF_NASCIMENTO']
del candidatos_2018['CD_MUNICIPIO_NASCIMENTO']
#del candidatos_2018['NM_MUNICIPIO_NASCIMENTO']
#del candidatos_2018['NR_IDADE_DATA_POSSE']
#del candidatos_2018['NR_TITULO_ELEITORAL_CANDIDATO']
del candidatos_2018['CD_GENERO']
#del candidatos_2018['DS_GENERO']
del candidatos_2018['CD_GRAU_INSTRUCAO']
#del candidatos_2018['DS_GRAU_INSTRUCAO']
del candidatos_2018['CD_ESTADO_CIVIL']
#del candidatos_2018['DS_ESTADO_CIVIL']
del candidatos_2018['CD_COR_RACA']
#del candidatos_2018['DS_COR_RACA']
del candidatos_2018['CD_OCUPACAO']
#del candidatos_2018['NR_DESPESA_MAX_CAMPANHA']
del candidatos_2018['CD_SIT_TOT_TURNO']
#del candidatos_2018['DS_SIT_TOT_TURNO']
del candidatos_2018['NR_PROTOCOLO_CANDIDATURA']
del candidatos_2018['NR_PROCESSO ']
del candidatos_2018['NM_SOCIAL_CANDIDATO']
#del candidatos_2018['DS_SITUACAO_CANDIDATURA'] # opcional
#del candidatos_2018['ST_REELEICAO'] # opcional
#del candidatos_2018['ST_DECLARAR_BENS'] # opcional

candidatos_2018.columns = ['ano_eleicao',
'uf', 
'cargo', 
'n_sequencial', 
'numero_urna',
'nome_completo', 
'nome_urna', 
'cpf', 
'e_mail', 
'situacao_candidatura', 
'partido_eleicao', 
'nacionalidade', 
'uf_nascimento', 
'nome_municipio_nascimento',  
'data_nascimento', 
'idade_data_posse', 
'titulo_eleitoral', 
'genero', 
'grau_instrucao', 
'estado_civil', 
'cor_raca', 
'ocupacao', 
'despesa_max_campanha_informada', 
'situcao_eleicao', 
'tentou_reeleicao', 
'declarou_bens']

candidatos_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ano_eleicao                     29145 non-null  object
 1   uf                              29145 non-null  object
 2   cargo                           29145 non-null  object
 3   n_sequencial                    29145 non-null  object
 4   numero_urna                     29145 non-null  object
 5   nome_completo                   29145 non-null  object
 6   nome_urna                       29145 non-null  object
 7   cpf                             29145 non-null  object
 8   e_mail                          29145 non-null  object
 9   situacao_candidatura            29145 non-null  object
 10  partido_eleicao                 29145 non-null  object
 11  nacionalidade                   29145 non-null  object
 12  uf_nascimento                   29145 non-null

In [21]:
#candidatos_2018["nome_completo"] = candidatos_2018["nome_completo"].apply(f)

In [22]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'latin_1'}
candidatos_2016 = pd.read_csv("../candidatos_2016/consulta_cand_2016_BRASIL.csv", **kwargs)

In [23]:
# Apaga as colunas que não serão usadas
del candidatos_2016['DT_GERACAO']
del candidatos_2016['HH_GERACAO']
#del candidatos_2016['ANO_ELEICAO']
del candidatos_2016['CD_TIPO_ELEICAO']
del candidatos_2016['NM_TIPO_ELEICAO']
del candidatos_2016['NR_TURNO']
del candidatos_2016['CD_ELEICAO']
del candidatos_2016['DS_ELEICAO']
del candidatos_2016['DT_ELEICAO']
del candidatos_2016['TP_ABRANGENCIA']
del candidatos_2016['SG_UE']
del candidatos_2016['NM_UE']
del candidatos_2016['CD_CARGO']
del candidatos_2016['CD_SITUACAO_CANDIDATURA']
del candidatos_2016['CD_DETALHE_SITUACAO_CAND']
del candidatos_2016['DS_DETALHE_SITUACAO_CAND']
del candidatos_2016['TP_AGREMIACAO']
del candidatos_2016['NR_PARTIDO']
del candidatos_2016['NM_PARTIDO']
del candidatos_2016['SQ_COLIGACAO']
del candidatos_2016['NM_COLIGACAO']
del candidatos_2016['DS_COMPOSICAO_COLIGACAO']
del candidatos_2016['CD_NACIONALIDADE']
#del candidatos_2016['DS_NACIONALIDADE']
#del candidatos_2016['SG_UF_NASCIMENTO']
del candidatos_2016['CD_MUNICIPIO_NASCIMENTO']
#del candidatos_2016['NM_MUNICIPIO_NASCIMENTO']
#del candidatos_2016['NR_IDADE_DATA_POSSE']
#del candidatos_2016['NR_TITULO_ELEITORAL_CANDIDATO']
del candidatos_2016['CD_GENERO']
#del candidatos_2016['DS_GENERO']
del candidatos_2016['CD_GRAU_INSTRUCAO']
#del candidatos_2016['DS_GRAU_INSTRUCAO']
del candidatos_2016['CD_ESTADO_CIVIL']
#del candidatos_2016['DS_ESTADO_CIVIL']
del candidatos_2016['CD_COR_RACA']
#del candidatos_2016['DS_COR_RACA']
del candidatos_2016['CD_OCUPACAO']
#del candidatos_2016['NR_DESPESA_MAX_CAMPANHA']
del candidatos_2016['CD_SIT_TOT_TURNO']
#del candidatos_2016['DS_SIT_TOT_TURNO']
del candidatos_2016['NR_PROTOCOLO_CANDIDATURA']
del candidatos_2016['NR_PROCESSO ']
del candidatos_2016['NM_SOCIAL_CANDIDATO']
#del candidatos_2016['DS_SITUACAO_CANDIDATURA'] # opcional
#del candidatos_2016['ST_REELEICAO'] # opcional
#del candidatos_2016['ST_DECLARAR_BENS'] # opcional

candidatos_2016.columns = ['ano_eleicao',
'uf', 
'cargo', 
'n_sequencial', 
'numero_urna',
'nome_completo', 
'nome_urna', 
'cpf', 
'e_mail', 
'situacao_candidatura', 
'partido_eleicao', 
'nacionalidade', 
'uf_nascimento', 
'nome_municipio_nascimento',  
'data_nascimento', 
'idade_data_posse', 
'titulo_eleitoral', 
'genero', 
'grau_instrucao', 
'estado_civil', 
'cor_raca', 
'ocupacao', 
'despesa_max_campanha_informada', 
'situcao_eleicao', 
'tentou_reeleicao', 
'declarou_bens']

candidatos_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498302 entries, 0 to 498301
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ano_eleicao                     498302 non-null  object
 1   uf                              498302 non-null  object
 2   cargo                           498302 non-null  object
 3   n_sequencial                    498302 non-null  object
 4   numero_urna                     498302 non-null  object
 5   nome_completo                   498302 non-null  object
 6   nome_urna                       498302 non-null  object
 7   cpf                             498302 non-null  object
 8   e_mail                          498302 non-null  object
 9   situacao_candidatura            498302 non-null  object
 10  partido_eleicao                 498302 non-null  object
 11  nacionalidade                   498302 non-null  object
 12  uf_nascimento                 

In [24]:
frames = [candidatos_2016, candidatos_2018]

In [25]:
candidatos_2016_2018 = pd.concat(frames)

In [26]:
candidatos_2016_2018 = candidatos_2016_2018.drop_duplicates(['cpf'], keep='last')

In [27]:
candidatos_2016_2018['cpf_mascarado']='***'+candidatos_2016_2018['cpf'].str[3:9]+'**'

In [28]:
candidatos_2016_2018[['nome_completo','cpf_mascarado']].head()

,nome_completo,cpf_mascarado
0,DINIFER DE ALMEIDA SEHN,***689969**
1,DIEGO DIAS,***173330**
2,CLAUDEMIR TACÍLIO GONÇALVES,***327329**
4,IRACY AMARAL NEGRE,***308221**
5,GELSON LUIZ PETRY,***620609**


In [29]:
# Verifica que empresas como sócios politicos brasileiros

In [30]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/setembro_2020/cnpj_dados_socios_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'identificador_socio',
                                      'nome_socio',
                                      'cnpj_cpf_socio',
                                      'cod_qualificacao_socio',
                                      'data_entrada_sociedade',
                                      'cpf_representante_legal',
                                      'nome_representante'])

In [31]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    socios_encontrados = pd.merge(candidatos_2016_2018, \
                                 df, \
                                 left_on=['nome_completo', 'cpf_mascarado'], \
                                 right_on=['nome_socio','cnpj_cpf_socio'])
    
    if socios_encontrados.empty is False:
        if conta == 0:
            df_procura_socio = socios_encontrados
        else:
            df_procura_socio = df_procura_socio.append(socios_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-10-20 16:59:21.089880
A extração terminou às:  2020-10-20 19:47:41.949320


In [32]:
df_procura_socio.shape

(172793, 35)

In [33]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/setembro_2020/cnpj_dados_cadastrais_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'razao_social',
                                      'nome_fantasia',
                                      'situacao_cadastral',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio'])

In [34]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    empresas_encontradas_polit = pd.merge(df_procura_socio, \
                                 df, \
                                 left_on= 'cnpj', \
                                 right_on='cnpj')
    
    if empresas_encontradas_polit.empty is False:
        if conta == 0:
            df_procura_polit = empresas_encontradas_polit
        else:
            df_procura_polit = df_procura_polit.append(empresas_encontradas_polit)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-10-20 20:26:08.609804
A extração terminou às:  2020-10-20 23:06:19.870465


In [35]:
df_procura_polit.shape

(172793, 43)

In [36]:
df_procura_polit.to_csv('resultados/empresas_politicos_candidatos_2016_2018_versao_15-out-20.csv', index=False)

In [37]:
# Carrega arquivo de empresas de políticos já salvo

In [38]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
df_procura_polit = pd.read_csv("resultados/empresas_politicos_candidatos/empresas_politicos_candidatos_2016_2018_versao_28-05-20.csv", **kwargs)

In [20]:
# Verifica se há empresas de políticos entre os fornecedores da covid19

In [39]:
empresas_encontradas_polit_covid19 = pd.merge(pagamentos_covid19.drop_duplicates('Código Favorecido_x'), \
                                 df_procura_polit, \
                                 left_on= 'Código Favorecido_x', \
                                 right_on='cnpj')

In [40]:
empresas_encontradas_polit_covid19.shape

(79, 146)

In [41]:
empresas_encontradas_polit_covid19[["uf_x", "nome_urna", "cargo", "situcao_eleicao", "Data Emissão_x","cod_empenho", "id_empenho", "Órgão_x", "Unidade Gestora_x", "Código Favorecido_x", "Favorecido_x", "Valor do Empenho Convertido pra R$", "valor_pago_em_real", "Valor Restos a Pagar Inscritos (R$)", "Observação_x", "Observação_y",  "Modalidade de Licitação"]]

,uf_x,nome_urna,cargo,situcao_eleicao,Data Emissão_x,cod_empenho,id_empenho,Órgão_x,Unidade Gestora_x,Código Favorecido_x,Favorecido_x,Valor do Empenho Convertido pra R$,valor_pago_em_real,Valor Restos a Pagar Inscritos (R$),Observação_x,Observação_y,Modalidade de Licitação
0,TO,OGARI PACHECO,2º SUPLENTE,ELEITO,02/06/2020,153152152362020NE800944,177003035,Universidade Federal do Rio de Janeiro,HOSPITAL UNIVERSITARIO DA UFRJ,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,"198240,00","-2,365.00","0,00",FARMACIA - AQUISICAO DE MATERIAL HOSPITALAR (D...,ERRO.,Pregão
1,DF,FERNANDO MARQUES,SENADOR,NÃO ELEITO,02/06/2020,153152152362020NE800957,173502971,Universidade Federal do Rio de Janeiro,HOSPITAL UNIVERSITARIO DA UFRJ,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,"28166,25",619.66,"0,00",FARMACIA - AQUISICAO DE MEDICAMENTOS (DEMANDA ...,"DEDUCOES DE TRIBUTOS FEDERAIS DA NF.422014,EMI...",Pregão
2,PR,ROSANA MONNEY RIBAS,DEPUTADO ESTADUAL,SUPLENTE,28/07/2020,155020264432020NE801998,237300204,Empresa Brasileira de Serviços Hospitalares,EBSERH COMPLEXO HOSPITALAR DO CEARÁ,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"1017,00",59.49,"0,00",SOLICITANTE UNIDADE DE PRODUTOS PARA SAUDE-MEA...,"RETENCAO DE TRIBUTOS FEDERAIS, REF. NF 1841, D...",Pregão
3,RS,LINO ZINN,VEREADOR,SUPLENTE,29/07/2020,155009264432020NE802060,239500340,Empresa Brasileira de Serviços Hospitalares,EBSERH HUB-UNB,04183656000148,DISTRICENTER DISTRIBUIDORA DE MEDICAMENTOS LTDA,"177888,00","3,021.48","0,00","ATENDER DESPESA COM MATERIAL FARMACOLOGICO,CON...",REC. NFE 5050 EMISSAO 30/07/2020 ATESTE 05/08/...,Dispensa de Licitação
4,MS,JAIRO ARRUDA,DEPUTADO FEDERAL,SUPLENTE,02/09/2020,160132000012020NE800522,230402086,Comando do Exército,9 BATALHAO DE ENGENHARIA DE COMBATE,07530947000153,COOPERATIVA DOS PRODUTORES RURAIS DA REGIAO DO...,"150,80",150.80,"0,00",AQUISICAO DE GENEROS ALIMENTICIOS * CONF 2020N...,"AQUISICAO DE MATERIAL DE CONSUMO PARA A OM, CO...",Dispensa de Licitação
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,MG,DEIR CASSIMIRO,VEREADOR,SUPLENTE,23/07/2020,135562222112020NE000026,241300021,Companhia Nacional de Abastecimento,UNIDADE CONAB PAA MINAS GERAIS,04774913000116,ASSOCIACAO COMUNITARIA DO CORREGO DOS FERREIRA...,"79968,62","75,290.46","0,00",EMPENHO DESTINADO A COMPRA C/ DOACAO SIMULTANE...,AQUISICAO DE PRODUTORES DA AGRICULTURA FAMILIA...,Dispensa de Licitação
75,CE,ZILANDA SIQUEIRA,VEREADOR,NÃO ELEITO,23/07/2020,550009000012020NE801325,235400066,Ministério da Cidadania - Unidades com víncul,SECR. NAC. DE CUIDADOS E PREV. AS DROGAS,07741268000123,ASSOCIACAO AGAPE,"42200,28","19,630.80","0,00",IMPORTANCIA QUE SE EMPENHA DESTINADO A ATENDER...,"PGTO. A ASSOCIACAO AGAPE, PELA PRESTACAO DE SE...",Inexigível
76,MG,PRETO DO ZÉ VICENTE,VEREADOR,NÃO ELEITO,23/07/2020,135562222112020NE000029,239900057,Companhia Nacional de Abastecimento,UNIDADE CONAB PAA MINAS GERAIS,11326035000131,ASSOCIACAO COMUNITARIA DO CORREGO DOS OLIVEIRAS,"184000,00","173,236.00","0,00",EMPENHO DESTINADO A COMPRA C/ DOACAO SIMULTANE...,AQUISICAO DE PRODUTORES DA AGRICULTURA FAMILIA...,Dispensa de Licitação
77,PE,OZANEIDE GOMES DO MANDACARU,VEREADOR,SUPLENTE,14/07/2020,135563222112020NE000034,206901336,Companhia Nacional de Abastecimento,UNIDADE CONAB PAA PERNAMBUCO,04411965000128,ASSOCIACAO DOS AGRICULTORES FAMILIARES DO ASSE...,"99662,90","93,832.62","0,00",CPR-PE-2020/02/0056 - DESTINADO A COMPRA COM D...,"PAGAMENTO DA CPR/PE/2020/02/0056, EM FAVOR DA ...",Não se Aplica


In [42]:
empresas_encontradas_polit_covid19.to_csv('resultados/pagamentos_covid19_empresas_candidatos/pagamentos_feitos_de_empenhos_covid19_empresas_politicos_ate_15_out_2020.csv', index=False)

In [43]:
# Checa no arquivo original de pagamentos e também para pegar os totais de cada

In [44]:
# AMAZONAS ENERGIA S.A

In [45]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160137000012020NE800064') &  
                           (pagamentos_covid19['id_empenho'] == '177600344')]

In [46]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
5540,02341467000120,AMAZONAS ENERGIA S.A,106.34
5541,02341467000120,AMAZONAS ENERGIA S.A,678.16
5542,02341467000120,AMAZONAS ENERGIA S.A,"1,715.50"


In [47]:
procura['valor_pago_em_real'].sum()

2500.0

In [48]:
# PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA

In [49]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801608') &  
                           (pagamentos_covid19['id_empenho'] == '183900342')]

In [50]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6114,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"3,060.00"


In [51]:
# CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA 

In [52]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801615') &  
                           (pagamentos_covid19['id_empenho'] == '174100368')]

In [53]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6059,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,17.55
6060,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,679.95


In [54]:
procura['valor_pago_em_real'].sum()

697.5

In [55]:
# UNIAO QUIMICA FARMACEUTICA NACIONAL S A 

In [56]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801652') &  
                           (pagamentos_covid19['id_empenho'] == '174200300')]

In [57]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6189,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,990.00
6190,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,"44,010.00"


In [58]:
procura['valor_pago_em_real'].sum()

45000.0

In [59]:
# FUNDO MUNICIPAL DE SAUDE DE RIO QUENTE

In [60]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '257001000012020NE818845') &  
                           (pagamentos_covid19['id_empenho'] == '180201534')]

In [61]:
procura[["Data Emissão_x", "Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Data Emissão_x,Código Favorecido_x,Favorecido_x,valor_pago_em_real
21185,04/05/2020,08278113000165,FUNDO MUNICIPAL DE SAUDE DE RIO QUENTE,"430,000.00"


In [62]:
# COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ELETRICA

In [63]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160388000012020NE800142') &  
                           (pagamentos_covid19['id_empenho'] == '174500711')]

In [64]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
53280,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,17.00
53281,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,273.69
53282,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,10.37
53283,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,166.90
53284,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,6.01
53285,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,96.80
53286,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,1.44
53287,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,23.19
53288,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,6.20
53289,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,100.10


In [65]:
procura['valor_pago_em_real'].sum()

1980.77

In [66]:
# VWX CONSTRUCOES E SERVICOS EIRELI

In [67]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '194048192082020NE800066') &  
                           (pagamentos_covid19['id_empenho'] == '179700716')]

In [68]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
53583,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"7,565.68"
53584,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"8,489.58"
53585,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"2,888.72"
53586,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"7,565.68"
53587,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"8,345.15"
53588,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"3,033.15"


In [69]:
procura['valor_pago_em_real'].sum()

37887.96

In [70]:
# RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS AUTOMOTIVO

In [71]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160055000012020NE800042') &  
                           (pagamentos_covid19['id_empenho'] == '144306058')]

In [72]:
procura[["Data Emissão_x", "Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Data Emissão_x,Código Favorecido_x,Favorecido_x,valor_pago_em_real
74684,13/03/2020,08990041000184,RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS A...,499.29
74685,13/03/2020,08990041000184,RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS A...,"9,486.44"


In [73]:
procura['valor_pago_em_real'].sum()

9985.730000000001

In [74]:
# SIQUEIRA TUR TRANSPORTE E TURISMO LTDA
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '194045192082020NE800087') &  
                           (pagamentos_covid19['id_empenho'] == '181302380')]
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
47578,05929516000139,SIQUEIRA TUR TRANSPORTE E TURISMO LTDA,"2,400.00"


In [75]:
procura['valor_pago_em_real'].sum()

2400.0

In [76]:
# JHONATHAN MACIEL DE SOUZA EIRELI

In [77]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160005000012020NE800119') &  
                           (pagamentos_covid19['id_empenho'] == '176401476')]

In [78]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
21958,31783115000123,JHONATHAN MACIEL DE SOUZA EIRELI,"2,814.00"
21959,31783115000123,JHONATHAN MACIEL DE SOUZA EIRELI,"2,814.00"
21960,31783115000123,JHONATHAN MACIEL DE SOUZA EIRELI,"-2,814.00"


In [79]:
procura['valor_pago_em_real'].sum()

2814.0

In [80]:
# E também depois no site da Receita: https://servicos.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp

In [81]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801608') &  
                           (pagamentos_covid19['id_empenho'] == '183900342')]

In [82]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6114,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"3,060.00"


In [83]:
procura['valor_pago_em_real'].sum()

3060.0